# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Обзор данных

Составим первое впечатление о таблице с данными:

In [44]:
# импортируем библиотеку pandas
import pandas as pd

# читаем и сохраняем файл с данными в df
df = pd.read_csv('/datasets/data.csv')

# выводим на экран первые 10 строк таблицы для ознакомления
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [45]:
# получим общую информацию о таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов.

Согласно документации к данным:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Количество значений в стобцах различается, значит в данных есть пропуски.

**Вывод**

В каждой строке таблицы находится информация о клиентах, которым ранее был одобрен кредит. Для того, чтобы далее работать с данными и проверять гипотезы, данные необходимо обработать: устранить пропуски и дубликаты (если они имеются), заменить некорректные значения и типы данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

В столбцах days_employed (общий трудовой стаж в днях) и total_income (ежемесячный доход) имеются пропущенные значения. 

In [46]:
df[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [47]:
# посчитаем количество пропущенных значений в колонке total_income
count_missing_value = df[df['days_employed'].isna()]['total_income'].isna().count()
print(count_missing_value)

2174


In [48]:
# посчитаем какой процент пропущенные строки занимают в общем массиве данных
missing_value = count_missing_value / len(df)
print(f"Пропущенные значения составляют {missing_value:.0%} данных")

Пропущенные значения составляют 10% данных


Данные в столбцах days_employed и total_income пропущенны в одних и тех же строках, и эти строки составляют 10% от всего массива данных, значит удалять их нежелательно. 
Следовательно эти пропуски необходимо заполнить. 

Есть гипотеза, что ежемесячный доход и стаж зависит от типа занятости. Сгруппируем столбцы и найдем средние значения/медиану по ним, чтобы заполнить пропуски наиболее корректными значениями.

In [49]:
# сгруппируем столбцы по типу занятости, для нахождения среднего значения и медианы по графе ежемесячный доход
df.groupby('income_type').agg({'total_income':['mean', 'median']})

total_income               
                          mean         median
income_type                                  
безработный      131339.751676  131339.751676
в декрете         53829.130729   53829.130729
госслужащий      170898.309923  150447.935283
компаньон        202417.461462  172357.950966
пенсионер        137127.465690  118514.486412
предприниматель  499163.144947  499163.144947
сотрудник        161380.260488  142594.396847
студент           98201.625314   98201.625314

Судя по таблице, по некоторым типам занятости среднее значение заработной платы равно медианному, а по другим типам - медианное значение меньше среднего (но ни в одном случае не наоборот). Так как среднее значение больше подвержено искажению ввиду выбросов - заполним пропущенные значения в графе 'income_type' медианой по типам занятости.

In [50]:
# выведем значение пропусков по столбцу total_income до обработки
print('Пропуски до обработки:', df['total_income'].isna().sum())

# напишем цикл, который меняет пустые значения в графе total_income на медиану, по типу занятости
for income_type in df['income_type'].unique():
    median = df.loc[df['income_type'] == income_type, 'total_income'].median()
    df.loc[(df['total_income'].isna()) & (df['income_type'] == income_type), 'total_income'] = median
    
print('Пропуски после обработки:', df['income_type'].isna().sum())



Пропуски до обработки: 2174
Пропуски после обработки: 0


В графе 'days_employed' встречаются отрицательные значения - заменим их на положительные, равные по модулю. И переведем трудовой стаж в количество лет, чтобы проверить, не выбиваются ли еще какие-то данные.

In [51]:
# заменяем отрицателные значения в столбце 'days_employed'на положительные
df['days_employed'] = df['days_employed'].abs()

# добавим в таблицу столбец с переводом трудового стажа из дней в годы
df['years_employed'] = df['days_employed']/365

# выведем первые 5 строк нового столбца на экран и его максимальное значение
print(df['years_employed'].head())
print(df['years_employed'].max())

0     23.116912
1     11.026860
2     15.406637
3     11.300677
4    932.235814
Name: years_employed, dtype: float64
1100.6997273296713


Очевидно, что в данных есть ошибка, т.к. трудовой стаж человека не может быть равен 1100 годам. Можем предположить, что данные в таблицу попали из разных источников, и часть из них была введена не в часах, а в днях. Т.к. трудовой стаж человека в годах редко превышает 45 лет, предположим, что все значения, которые больше этого числа - были введены в часах, а не сутках, и заменим их.

In [52]:
print('Среднее значение трудового стажа в годах до обработки',df['years_employed'].mean())

# переведем в сутки данные, трудовой стаж которых был введен ранее в часах
df.loc[df['years_employed'] > 45, 'years_employed'] = df.loc[df['years_employed'] > 45, 'years_employed']/24

print('Среднее значение трудового стажа в годах после обработки',df['years_employed'].mean())
   

Среднее значение трудового стажа в годах до обработки 183.32802440225305
Среднее значение трудового стажа в годах после обработки 12.70968857183236


Теперь в столбце со стажем работы не осталось отрицательных значений и выборосов, стаж переведен в часы. Но остались строки с пустыми значениями - заполним их средним значением по типу занятости.

In [53]:
# выведем значение пропусков по столбцу years_employed до обработки
print('Пропуски до обработки:', df['years_employed'].isna().sum())

# напишем цикл, который меняет пустые значения в графе total_income на медиану, по типу занятости
for income_type in df['income_type'].unique():
    median = df.loc[df['income_type'] == income_type, 'years_employed'].median()
    df.loc[(df['years_employed'].isna()) & (df['income_type'] == income_type), 'years_employed'] = median
    
print('Пропуски после обработки:', df['income_type'].isna().sum())

# заполним значения в колонце days_employed на корректные
df['days_employed'] = df['years_employed']*365


Пропуски до обработки: 2174
Пропуски после обработки: 0


In [54]:
# выведем информацию о таблице, чтобы убедиться, что в ней не осталось пропусков
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
 12  years_employed    21525 non-null  float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.1+ MB


**Вывод**

В таблице имелись пропущенные данные в столбцах:
- с ежемесячным доходом (заменили пустоты на медианное значение дохода по типу занятости)
- со стажем работника (заменили отрицательные значения на положительные, пустоты заменили на медианное значение дохода по типу занятости)

После обработки в таблице не осталось пропущенных значений, но имеются колонки, в которых необходимо заменить тип данных.

### Замена типа данных

В таблице имеются колонки days_employed, years_employed, total_income с вещественным типом данных, это не совсем корректно/удобно, т.к. дни и годы в жизни определяются целыми числами - переведем их в целочесленный тип данных int, использую метод astype.

In [55]:
# перевод значений в столбцах days_employed и years_employed в целочисенный тип данных
df['days_employed'] = df['days_employed'].astype(int)
df['years_employed'] = df['years_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)

# вывод информации о данных столбцах
df.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
years_employed       int64
dtype: object

**Вывод**

Все столцы теперь имеют корректный тип данных для работы с ними и можно переходить к обработке дубликатов.

### Обработка дубликатов

Проверим, имеются ли в табице дубликаты.

In [56]:
# проверка дубликатов в таблице
print("Дубликатов в таблице:", df.duplicated().sum())

Дубликатов в таблице: 54


In [57]:
# посмотрим на строчки, где встречаются дубликаты
df[df.duplicated()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
2849,0,1573,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи,4
4182,1,1573,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594,свадьба,4
4851,0,15217,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба,41
5557,0,15217,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу,41
7808,0,15217,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,41
8583,0,15217,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование,41
9238,2,1573,34,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для сдачи,4
9528,0,15217,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью,41
9627,0,15217,56,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции со своей недвижимостью,41
10462,0,15217,62,среднее,1,женат / замужем,0,F,пенсионер,0,118514,покупка коммерческой недвижимости,41


В стобце education есть данные, которые написаны в разных регистрах, следовательно они не попадают в категории дубликаты, хотя должны. Проверим все уникальные значения в столбцах, имеющих строковый тип данных, чтобы, при необходимости перевести данные в нижний регистр и удалить все дубликаты.

In [58]:
# выведем уникальные значения для столбца education
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [59]:
# выведем уникальные значения для столбца gender
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [60]:
# выведем уникальные значения для столбца income_type
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [61]:
# выведем уникальные значения для столбца family_status
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [62]:
# выведем уникальные значения для столбца purpose
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

После проверки уникальных значений во всех столбцах со строковыми данными видим, что необходимость к переводу данных в один регистр имеется только для столбца education. После этого еще раз проверим количество дубликатов (их может стать больше) и удалим их методом drop_duplicates.

Для столбца purpose требуется лемматизация (исполним в следующем блоке). 

In [63]:
# переведем все значения столбца education в нижний регистр
df['education'] = df['education'].str.lower()

# проверим уникальные значения для столбца education
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

В столбце gender имеется значение XNA, которое не относится ни мужскому, ни к женсому полу - удалим строку с этим значением, чтобы она не мешала дальнейшей обработке.

In [64]:
# удалим строку со значение гендера XNA
df = df.drop(df[df['gender'] == 'XNA'].index)


Проверим на аномалии столбцы с числовыми значениями

In [65]:
# проверим уникальные значения для столбца education
df['children'].value_counts()

 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце есть значение "20" детей, что врядли соответствует истине. 
Также не может быть -1 ребенок. Посчитаем долю некорректных значений, и если она меньше 5% - удалим эти строки.


In [66]:
# посчитаем долю некорреткных значений
(len(df[df['children'] == 20])+len(df[df['children'] == -1]))/len(df)*100

0.5714551198661958

In [67]:
# удаляем строки с 20 и -1 детьми.
df = df.drop(df[df['children'] == 20].index)
df = df.drop(df[df['children'] == -1].index)

In [68]:
# проверка дубликатов в таблице
print("Дубликатов в таблице:", df.duplicated().sum())

Дубликатов в таблице: 71


In [69]:
# удалим дубликаты в таблице
df = df.drop_duplicates().reset_index(drop=True)

# проверка дубликатов в таблице после удаления
print("Дубликатов в таблице после:", df.duplicated().sum())

Дубликатов в таблице после: 0


**Вывод**

В таблице с исходными данными имелись явные дубликаты в столбце с образованием клиента. После приведения значений из этого столбца к нижниему регистру - удлось избавиться от всех явных дубликатов.


### Лемматизация

В столбце purpose имеются значения с однокоренными словами, которые необходимо обработать, для дальнейшей категоризации и анализа. Исползуем метод лемматизации.

Получим уникальные значения столбца purpose, склеим их в одну строку и лемматизируем с помощью библиотеки PyMystem.

In [70]:
# получим уникальные значения столбца purpose и в всиде строки присвоим их переменной unique_purpose_str
unique_purpose_str = ' '.join(df['purpose'].unique())

# импортируем библиотеку pymystem3 для лемматизаци
from pymystem3 import Mystem
m = Mystem()
lemmas = m.lemmatize(unique_purpose_str)

# посчитаем, сколько раз встречаются уникальные слова с переменной unique_purpose_str
from collections import Counter
print(Counter(lemmas))

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


Чаще всего в полученном списке встречается слово покупка, но оно может относиться, как к автомобилю, так и к недвижимости, поэтому для дальнейшей категоризации не подойдет. Жильё можно условно приравнять к недвижимости. Слова: строительство, получение, сдача, ремонт также относятся к недвижимости.

Таким образом, из списка получится всего 4 категории: недвижимость, автомобиль, образование и свадьба.

Напишем функцию, которая будет лематизировать последовательно каждую ячейку столбца purpose и проверять, к какой категрии относится каждая строка.

In [71]:
def lemmas_purpose(purpose):
    purpose_lemmas = m.lemmatize(purpose)
    if 'недвижимость' in purpose_lemmas or 'жилье' in purpose_lemmas:
        return 'недвижимость'
    if 'автомобиль' in purpose_lemmas:
        return 'автомобиль'
    if 'образование' in purpose_lemmas:
        return 'образование'
    if 'свадьба' in purpose_lemmas:
        return 'свадьба'

# проверим, как работает функция
lemmas_purpose('ремонт жилью')


'недвижимость'

Создадим в таблице столбец lemmas_purpose с лемматизированными данными по столбцу purpose и проверим, все ли значения в этом стобце попали под ту или иную категорию.

In [72]:
df['lemmas_purpose'] = df['purpose'].apply(lemmas_purpose)

print('Количество пропусков в стобце',df['lemmas_purpose'].isna().sum())

df['lemmas_purpose'].value_counts()

Количество пропусков в стобце 0


недвижимость    10750
автомобиль       4279
образование      3988
свадьба          2313
Name: lemmas_purpose, dtype: int64

**Вывод**

В столбце purpose данные записаны разными спосабами, что может помешать дальнейшей категоризации. Для того, чтобы устранить эту проблему, мы, с помощью лемматизации, создали функцию, которая обрабатывает данные этого столбца и вносит новые данные в столбец lemmas_purpose.

Теперь можно перейти к категоризации данных по другим столбцам.

### Категоризация данных

Для того, чтобы быстро и качественно отвечать на вопросы бизнеса, наши данные должны быть структурированы и разбиты на категории.

Один из вопросов, на которые в последствии нам нужно будет ответить: "есть ли зависимость между наличием детей и возвратом кредита в срок?" Значит нужно категоризировать данные по количеству детей. Но для начала проверим, корректные ли данные находятся в стобце children.

Добавим столбец having_children, который будет принимать значения 0 - если детей нет, и 1 - если дети есть (в любом количестве)

In [73]:
df.loc[df['children'] == 20,'children'] = 2
df.loc[df['children'] == -1, 'children'] = 0

def having_children(children):
    if children == 0:
        return 0
    else:
        return 1
    
df['having_children'] = df['children'].apply(having_children)


В таблице имеется 2 столбца education и education_id, данные в которых должны быть между собой связаны. Проверим, так ли это.

In [74]:
df[['education', 'education_id']].drop_duplicates().set_index('education_id')

,education
education_id,
0,высшее
1,среднее
2,неоконченное высшее
3,начальное
4,ученая степень


В столбцах, приведенных выше, нет расхождений между данными. Проверим таким же образом столбцы family_status и family_status_id.

In [75]:
df[['family_status', "family_status_id"]].drop_duplicates().set_index('family_status_id')

,family_status
family_status_id,
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,Не женат / не замужем


В столбцах, приведенных выше, также нет расхождений между данными.

Категоризируем уровень дохода на: меньше среднего, средний, выше среднего и занесем эти данные в таблицу income.

In [76]:
# присвоим переменным income_mean и income_median среднее и медианное значение столбца total_income соответственно
income_mean = df['total_income'].mean()
income_median = df['total_income'].median()

# напишем функцию, которая категоризирует уровень дохода
def income(total_income):
    if total_income < income_median:
        return "ниже среднего"
    if income_median <= total_income <= income_mean:
        return "средний"
    else:
        return "выше среднего"
    
# пишем столбец, который показывает категоризированный уровень дохода
df['income'] = df['total_income'].apply(income)

df['income'].value_counts()


ниже среднего    9717
выше среднего    8126
средний          3487
Name: income, dtype: int64

Категоризируем людей по возрасту, для этого посмотрим на данные по этому столбцу

In [77]:
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

В столбце имеются нулевые значения - заменим их на сумму трудового стажа в годах и 18 + 3 (возраст, когда человек после обучения в среднем начинает трудовую карьеру).

In [78]:
df.loc[df['dob_years'] == 0, 'dob_years'] = (df['years_employed'] + 18 + 3)

In [79]:
def age(dob_years):
    if dob_years <= 18:
        return "18"
    if 18 < dob_years <= 25:
        return "19-25"
    if 25 < dob_years <= 35:
        return "26-35"
    if 35 < dob_years <= 45:
        return "36-45"
    if 45 < dob_years <= 60:
        return "46-60"
    else:
        return "> 60"
    
df['age'] = df['dob_years'].apply(age)


In [80]:
# выведем первые 10 строк таблицы, чтобы проверить корректность данных и решить,
# всей ли информации нам хватает для провери гипотез
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,lemmas_purpose,having_children,income,age
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,недвижимость,1,выше среднего,36-45
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,автомобиль,1,ниже среднего,36-45
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,недвижимость,0,средний,26-35
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,образование,1,выше среднего,26-35
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,38,свадьба,0,средний,46-60
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,2,недвижимость,0,выше среднего,26-35
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,7,недвижимость,0,выше среднего,36-45
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,0,образование,0,ниже среднего,46-60
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,18,свадьба,1,ниже среднего,26-35
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,5,недвижимость,0,средний,36-45


**Вывод**

Мы категоризировали данные по количеству детей, семейному положению, уровню дохода, целей на получение кредита и образованию.

Можно приступать к проверке гипотез.

## Шаг 3. Проверка гипотез

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [81]:
# формируем сводную таблицу по столбцам having_children и children
# значения сводной таблицы берем из столбца debt
# применяем функции sum, count, mean
pivot_children = df.pivot_table(index=['having_children', 'children'], values='debt', aggfunc=['sum', 'count', 'mean'])

# приводим значение среднего к процентам
pivot_children['mean'] = (pivot_children['mean']*100).round(2)

# переименовываем столбцы
pivot_children.columns = ['has_debt', 'total', '%']

pivot_children

has_debt  total     %
having_children children                       
0               0             1063  14090  7.54
1               1              444   4808  9.23
                2              194   2052  9.45
                3               27    330  8.18
                4                4     41  9.76
                5                0      9  0.00

**Вывод**

Из таблицы следует, что люди с наличием детей(до 5), в среднем чаще имеют задолженость по кредиту, чем бездетные. Люди с 5 детьми задолженностей не имели вовсе, но т.к. их всего 9 человек(менее 0,04% от всей выборки) - считаю данный результат не показательным.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [82]:
# формируем сводную таблицу по столбцу family_status
# значения сводной таблицы берем из столбца debt
# применяем функции sum, count, mean
pivot_family_status = df.pivot_table(index=['family_status'], values='debt', aggfunc=['sum', 'count', 'mean'])

# приводим значение среднего к процентам
pivot_family_status['mean'] = (pivot_family_status['mean']*100).round(2)

# переименовываем столбцы
pivot_family_status.columns = ['has_debt', 'total', '% debt']

pivot_family_status['% total'] = (pivot_family_status['total']/pivot_family_status['total'].sum()*100).round(2)

# отсортируем данные по столбцу '%'
pivot_family_status = pivot_family_status.sort_values(by='% debt', ascending=False)

pivot_family_status

,has_debt,total,% debt,% total
family_status,,,,
Не женат / не замужем,273,2796,9.76,13.11
гражданский брак,385,4133,9.32,19.38
женат / замужем,927,12261,7.56,57.48
в разводе,84,1189,7.06,5.57
вдовец / вдова,63,951,6.62,4.46


**Вывод**

Из таблицы видно, что не женатые/не замужние люди, и люди, живущие в гражданском браке чаще имеют задолженности по кредитам, чем другие категории людей. Разведенные люди и вдовцы реже имеют кредитную задоженность. Возможно это как-то связано с возрастом, ведь разводят и вдовеют люди позже, чем женятся.

- Есть ли зависимость между возрастом человека и возвратом кредита в срок?

In [83]:
# формируем сводную таблицу по столбцу age
# значения сводной таблицы берем из столбца debt
# применяем функции sum, count, mean
pivot_age = df.pivot_table(index=['age'], values='debt', aggfunc=['sum', 'count', 'mean'])

# приводим значение среднего к процентам
pivot_age['mean'] = (pivot_age['mean']*100).round(2)

# переименовываем столбцы
pivot_age.columns = ['has_debt', 'total', '% debt']

pivot_age['% total'] = (pivot_age['total']/pivot_age['total'].sum()*100).round(2)

# отсортируем данные по столбцу '%'
pivot_age = pivot_age.sort_values(by='% debt', ascending=False)

pivot_age

,has_debt,total,% debt,% total
age,,,,
19-25,136,1279,10.63,6.00
26-35,566,5339,10.60,25.03
36-45,464,5586,8.31,26.19
46-60,465,6994,6.65,32.79
> 60,101,2132,4.74,10.00


**Вывод**

Людии более старшего возраста (от 36 лет) в среднем реже имеют кредитную задолженность. Значит гипотеза подтвердилась.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [84]:
# формируем сводную таблицу по столбцу income
# значения сводной таблицы берем из столбца debt
# применяем функции sum, count, mean
pivot_income = df.pivot_table(index=['income'], values='debt', aggfunc=['sum', 'count', 'mean'])

# приводим значение среднего к процентам
pivot_income['mean'] = (pivot_income['mean']*100).round(2)

# переименовываем столбцы
pivot_income.columns = ['has_debt', 'total', '% debt']

# отсортируем данные по столбцу '% debt'
pivot_income = pivot_income.sort_values(by='% debt', ascending=False)

pivot_income

,has_debt,total,% debt
income,,,
средний,304,3487,8.72
ниже среднего,809,9717,8.33
выше среднего,619,8126,7.62


**Вывод**

Люди, с уровем дохода выше среднего чаще возвращают кредит в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [85]:
# формируем сводную таблицу по столбцу purpose
# значения сводной таблицы берем из столбца debt
# применяем функции sum, count, mean
pivot_lemmas_purpose = df.pivot_table(index=['lemmas_purpose'], values='debt', aggfunc=['sum', 'count', 'mean'])

# приводим значение среднего к процентам
pivot_lemmas_purpose['mean'] = (pivot_lemmas_purpose['mean']*100).round(2)

# переименовываем столбцы
pivot_lemmas_purpose.columns = ['has_debt', 'total', '% debt']
pivot_lemmas_purpose['% total'] = (pivot_lemmas_purpose['total']/pivot_lemmas_purpose['total'].sum()*100).round(2)

# отсортируем данные по столбцу '% debt'
pivot_lemmas_purpose = pivot_lemmas_purpose.sort_values(by='% debt', ascending=False)

pivot_lemmas_purpose

,has_debt,total,% debt,% total
lemmas_purpose,,,,
автомобиль,400,4279,9.35,20.06
образование,369,3988,9.25,18.70
свадьба,183,2313,7.91,10.84
недвижимость,780,10750,7.26,50.40


**Вывод**

Люди, берущие кредит на недвижимость - чаще возвращают его в срок. Кредиты на автомобиль имеют больше всего рисков.

- Как уровень образования влияет на возврат кредита в срок?

In [86]:
# формируем сводную таблицу по столбцу education
# значения сводной таблицы берем из столбца debt
# применяем функции sum, count, mean
pivot_education = df.pivot_table(index=['education'], values='debt', aggfunc=['sum', 'count', 'mean'])

# приводим значение среднего к процентам
pivot_education['mean'] = (pivot_education['mean']*100).round(2)

# переименовываем столбцы
pivot_education.columns = ['has_debt', 'total', '% debt']
pivot_education['% total'] = (pivot_education['total']/pivot_education['total'].sum()*100).round(2)

# отсортируем данные по столбцу '% debt'
pivot_education = pivot_education.sort_values(by='% debt', ascending=False)

pivot_education

,has_debt,total,% debt,% total
education,,,,
начальное,31,282,10.99,1.32
неоконченное высшее,68,740,9.19,3.47
среднее,1355,15075,8.99,70.68
высшее,278,5227,5.32,24.51
ученая степень,0,6,0.00,0.03


**Вывод**

Чем полнее у человека образования - тем больше вероятность возврата кредита в срок.

## Шаг 4. Общий вывод

Для того, чтобы анализировать данные в таблице - мы их обработали, а именно:
- В столбцах days_employed (общий трудовой стаж в днях) и total_income (ежемесячный доход) имелись пропущенные значения - мы заменили их на медианные значения по типу занятости;
- Обнаружили, что в столбце days_employed - не все значения были проставлены в днях, имелись также значения и в часах. Добавили стобец years_employed, где привели все значения в корректную форму - трудовой стаж в годах;
- Для более корректной дальнейшей работы с данными, перевели из вещественного типа в целочисленные следующие столбцы: days_employed, years_employed, total_income;
- Данные столбца education перевели в нижний регистр, чтобы избежать задвоек;
- Обнаружили в графе children аномальные значения: 20 и -1 ребенок, и, т.к. эти значения составляли 0,5% от выборки - удалили данные строки;
- В столбце gender обнаружился пол XNA (1 строка), которые не относился ни к женскому, ни к мужскому - эту строку также удалили;
- Избавились от дубликатов в таблице;
- Выделили с помощью лемматизации 4 основные цели, на которые клиенты брали кредиты: недвижимость, свадьба, образование, автомобили;
- Заполнили нулевые значения в графе с возрастом, как "18 + 3 + стаж работы в годах" и поделили возраст на категории: до 18 лет, 19-25 лет, 26-35 лет, 36 - 45 лет, 46 - 60 лет, >60 лет;
- Распределили клиентов по 3 группам, в зависимости от уровня дохода.
    

После того, как даные были обработаны, мы проверили ряд гипотез и пришли к следующим выводам:
- люди в разводе и вдовцы чаще возвращают кредит в срок, возможно это связано с возрастом, т.к.
- люди старшего возраста (от 36 лет и более) более аккуратны при выполнении кредитных обязательств
- бездетные люди чаще соблюдают кредитные обязательство, чем люди с детьми;
- на недвижимость приходится 50% всех кредитов и выплачиваются они с наименьшими рисками, в отличает от кредитов на автомобили и образование.
- уровень образования также влияет на вероятность возврата кредита в срок: чем образование полнее, чем больше вероятность.

Таким образом, наиболее благоприятными клиентами являются лица старше 36 лет, с хорошим образование, которые берут кредит на недвижимость/свадьбу. Самая потенциально опасная группа - молодые люди, с начальным/средним/неоконченным высшим образованием, берущие кредит на автомобиль.